# CHAPTER 7 실습: 모니터링과 피드백 루프

이 노트북은 『MLOps 도입 가이드』 7장의 모니터링과 피드백 루프 실습을 위한 템플릿입니다.

실습 항목:
- 데이터 드리프트 감지 (KS 검정)
- 분포 시각화
- 개념 드리프트(모델 성능 변화) 탐지
- 드리프트 알림 시뮬레이션

## 1. 데이터 로딩 및 전처리

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np

# 타이타닉 데이터 로딩 및 전처리
df = sns.load_dataset('titanic')
df = df.dropna(subset=['fare', 'age', 'embarked', 'sex'])
df['sex'] = df['sex'].map({'male': 0, 'female': 1})
df['embarked'] = df['embarked'].astype('category').cat.codes


## 2. 분포 시각화 및 훈련/테스트 분리

In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# 70/30 분할
train, test = train_test_split(df, test_size=0.3, random_state=42)
# 분포 시각화
plt.hist(train['fare'], bins=20, alpha=0.5, label='train')
plt.hist(test['fare'], bins=20, alpha=0.5, label='test')
plt.legend(); plt.title('Fare Distribution')


## 3. 데이터 드리프트 감지 (KS 검정)

In [ ]:
from scipy.stats import ks_2samp

stat, p_value = ks_2samp(train['fare'], test['fare'])
print(f'KS 통계량: {stat:.4f}, p-value: {p_value:.4f}')
if p_value < 0.05:
    print('⚠️ 데이터 드리프트 감지됨')
else:
    print('✅ 드리프트 없음')

## 4. 개념 드리프트 감지: 모델 성능 변화

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

def evaluate_performance(data):
    X = data[['pclass','sex','age','sibsp','parch','fare','embarked']]
    y = data['survived']
    model = RandomForestClassifier(random_state=42)
    X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.3, random_state=42)
    model.fit(X_train, y_train)
    return accuracy_score(y_eval, model.predict(X_eval))

base_acc = evaluate_performance(train)
drift_acc = evaluate_performance(test)
print(f'기준 정확도: {base_acc:.4f}, 테스트 정확도: {drift_acc:.4f}')

## 5. 드리프트 알림 시뮬레이션

In [ ]:
drop = base_acc - drift_acc
if drop > 0.05:
    print('🚨 모델 성능 하락 감지: 경고 전송')
else:
    print('✅ 모델 성능 양호')

🔧 실제 운영에서는 이 알림을 이메일, Slack, 모니터링 시스템과 연동합니다.